In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Загрузка и препроцессинг данных

Нейросети не работают с сырыми даными по типу JPEG или CSV. Вместо этого они оперируют **векторами** и **специальными** представлениями:
* Текстовые файлы должны быть прочитаны, строки получены и разбиты на слова. Слова должны быть проиндексированы и превращены в целочисленные тензоры.
* Изображения должны быть так же прочитаны и преобразованы в тензоры пикселей (целочисленные), а затем нормализованы до некоторых маленьких значений (часто 0-1).
* Табличные данные должны быть обработаны: категоризованные признаки должны быть приведены в целочисленные тензоры, а признаки в виде чисел с плавающей точкой преобразованы в соответствующие тензоры с плавающей точкой. В конце все признаки точно так же должны быть нормализованы, т.к. этого требуют модели.
* Ну и т.д.

## Загрузка данных

Собственно, Keras оперирует тремя типами "входов" (или входных данных):
* NumPy массивами (те самые `ndarray`), точно так же, как это делает sklearn. Это хороший вариант, если все данные умещаются в RAM.
* Tensorflow Dataset объекты. Это вариант подходит для тех случаев, когда данные не помещаются в память и будут считывать либо с диска, либо вообще с распределенной ФС.
* Питоновские генераторы, которые отдают пакеты данных (очевидно,  всё, что реализовано как генератор, может передаваться в Keras).

Важно отметить, что если датасет правда большой, а вы собираетесь обучать модель на GPU, то стоит использовать объект Tensorflow Dataset, т.к. он обеспечивает высокую производительность засчёт:
* ассинхронной предобработки данных на CPU, пока GPU загружена, и постановка предобработанных данных в очередь
* поставки данных в GPU память так, что она немедленно становится доступна для GPU в момент, когда он прекратил обработку предыдущего поднабора данных. Это позволяет использовать GPU на полную.

Keras позволяет использовать ряд утилит для загрузки сырых данных с диска и превращения их сразу в объект `Dataset`:
* `tf.keras.preprocessing.image_dataset_from_directory` - для превращения изображений, отсортированных по специальной классовой структуре (в виде структуры папок) в размеченный датасет тензоров изображений.
* `tf.keras.preprocessing.text_dataset_from_directory` - то же самое, но для текстов

Как пример просто загрузим [вот этот датасет](https://www.kaggle.com/olgabelitskaya/tomato-cultivars) и распределим его по директориям (каждая директория представляет собой отдельный конкретный класс). Теперь можем воспользоваться методом выше, чтобы автоматически создать размеченный датасет из структуры папок.

Моя структура папок:
```
Tomato Cultivars/
...tomato_1/
......01_001.png
......01_002.png
................
...tomato_2/
......02_001.png
......02_002.png
................
............
...tomato_15/
......15_001.png
......15_002.png
................
```

In [8]:
main_directory = '/home/lpshkn/Datasets/Tomato Cultivars/'

In [9]:
dataset = keras.preprocessing.image_dataset_from_directory(main_directory)

Found 776 files belonging to 15 classes.


In [10]:
dataset.class_names

['tomato_1',
 'tomato_10',
 'tomato_11',
 'tomato_12',
 'tomato_13',
 'tomato_14',
 'tomato_15',
 'tomato_2',
 'tomato_3',
 'tomato_4',
 'tomato_5',
 'tomato_6',
 'tomato_7',
 'tomato_8',
 'tomato_9']

Как можно заметить, это идеально подходит для случаев, когда данных в виде текстовых или изображений очень много и все они разбросаны по директориям. Тогда можно написать простой bash скрипт, уложив всё по папкам, и использовать эти функции.

## Предобработка данных